# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [9]:
# Dependencies and Setup
import pandas as pd
import requests
import folium

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,egvekinot,66.3167,-179.1667,-26.80,87,100,3.90,RU,1675727310
1,1,bairiki,1.3292,172.9752,30.00,66,75,5.66,KI,1675727310
2,2,bambanglipuro,-7.9500,110.2833,24.07,93,100,1.89,ID,1675727311
3,3,yellowknife,62.4560,-114.3525,-20.71,77,75,3.60,CA,1675727201
4,4,saskylakh,71.9167,114.0833,-35.86,100,97,1.10,RU,1675727311


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [11]:
# Configure the map plot
mean_lat = city_data_df['Lat'].mean()
mean_lon = city_data_df['Lng'].mean()
Map_Plot = folium.Map(location=[mean_lat, mean_lon], zoom_start=3)

# Add a point for each city in the DataFrame
for index, row in city_data_df.iterrows():
    humidity = row['Humidity']
    folium.CircleMarker(location=[row['Lat'], row['Lng']],
                        popup=row['City'],
                        radius=humidity/20,
                        color='blue',
                        fill=True,
                        fill_color='blue',
                        fill_opacity=0.5).add_to(Map_Plot)

# Display the map
Map_Plot

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [12]:
# Narrow down cities that fit criteria and drop any results with null values
# Criteria of max temp between 21 and 27 degrees, wind speed below 4.5 and no clouds
filter_temp_df = city_data_df.loc[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27)]
filter_wind_df = filter_temp_df.loc[(filter_temp_df["Wind Speed"] < 5)]
Ideal_city_data_df = filter_wind_df.loc[(filter_wind_df["Cloudiness"] == 0)]

# Drop any rows with null values
Ideal_city_data_df = Ideal_city_data_df.dropna()
Ideal_city_data_df =Ideal_city_data_df.reset_index(drop=True)

# Display sample data
Ideal_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,30,vila velha,-20.3297,-40.2925,26.08,86,0,0.00,BR,1675727247
1,388,acajutla,13.5928,-89.8275,26.71,60,0,3.27,SV,1675727464
2,518,snyder,32.7179,-100.9176,25.19,20,0,4.63,US,1675727517


### Step 3: Create a new DataFrame called `hotel_df`.

In [13]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = Ideal_city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,vila velha,BR,-20.3297,-40.2925,86,
1,acajutla,SV,13.5928,-89.8275,60,
2,snyder,US,32.7179,-100.9176,20,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [14]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "filter":[],
    "bias":[],
    "limit": 1,
    "apiKey":geoapify_key    
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df.loc[index, "Lng"]
    latitude = hotel_df.loc[index, "Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
vila velha - nearest hotel: Hotel Prainha
acajutla - nearest hotel: Hotel Lara
snyder - nearest hotel: Days Inn


,City,Country,Lat,Lng,Humidity,Hotel Name
0,vila velha,BR,-20.3297,-40.2925,86,Hotel Prainha
1,acajutla,SV,13.5928,-89.8275,60,Hotel Lara
2,snyder,US,32.7179,-100.9176,20,Days Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [22]:
# Configure the map plot
Map_Plot_2 = folium.Map(location=[mean_lat, mean_lon], zoom_start=3)

# Add a point for each city in the DataFrame
for index, row in hotel_df.iterrows():
    folium.CircleMarker(location=[row['Lat'], row['Lng']],
                        popup="Country: "+row["Country"]+", Hotel Name:"+row["Hotel Name"],
                        color='blue',
                        fill=True,
                        fill_color='blue',
                        fill_opacity=0.5).add_to(Map_Plot_2)

Map_Plot_2